<a href="https://colab.research.google.com/github/DoitDigital-lab/Calculadora/blob/main/PocoVidToCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import sys
sys.path.append('/content/drive/My Drive/full_path')
sys.path.append('/content/drive/MyDrive/Pocovid/pocovidnet/')

Mounted at /content/drive/


In [ ]:
import argparse
import os
import gc
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from imutils import paths
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from pocovidnet import MODEL_FACTORY
from pocovidnet.utils import Metrics
from tensorflow.keras.models import Model
import os
from imutils import paths

In [ ]:
#@title Texto de título predeterminado
def naive(name):
	# Suppress logging
	tf.get_logger().setLevel('ERROR')
	# Initialize hyperparameters
	DATA_DIR = "/content/drive/MyDrive/Pocovid/data/cross_validation/"
	#DATA_DIR = "/content/drive/MyDrive/Pocovid/data/SuperC/"
	MODEL_NAME = str(name)
	FOLD = "4"
	MODEL_DIR = os.path.join('models/', MODEL_NAME, f'fold_4')
	LR = 1e-4
	EPOCHS = 30
	BATCH_SIZE = 32
	MODEL_ID = "vgg_base"
	TRAINABLE_BASE_LAYERS = 1
	IMG_WIDTH, IMG_HEIGHT = 224, 224
	LOG_SOFTMAX = True
	HIDDEN_SIZE = 64
	filter_types_applied = str(name)
	filter_types_applied_string=str(name);
	# Check if model class exists
	if MODEL_ID not in MODEL_FACTORY.keys():
	  raise ValueError(
		f'Model {MODEL_ID} not implemented. Choose from {MODEL_FACTORY.keys()}'
		)

	if not os.path.exists(MODEL_DIR):
		os.makedirs(MODEL_DIR)

	# grab the list of images in our dataset directory, then initialize
	# the list of data (i.e., images) and class images
	print('Loading images...')
	imagePaths = list(paths.list_images(DATA_DIR))
	data = []
	labels = []

	print(f'selected fold: {FOLD}')

	train_labels, test_labels = [], []
	train_data, test_data = [], []
	# test_files = []

	# loop over folds
	for imagePath in imagePaths:

		path_parts = imagePath.split(os.path.sep)
		#extract the split
		train_test = path_parts[-3][-1]
		# extract the class label from the filename
		label = path_parts[-2]
		image = cv2.imread(imagePath)
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		for filterType in filter_types_applied:
			if filterType == "0" :
				image = image
			if filterType == "1" :
				kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
				image = cv2.filter2D(image, -1, kernel)
			if filterType == "2" :
				ksize = (3, 3)
				image = cv2.blur(image, ksize)
			if filterType == "3" :
			  image = cv2.medianBlur(image,3)
			if filterType == "4" :
				image = cv2.Laplacian(image,cv2.CV_8U)
			if filterType == "5" :
				image = cv2.Sobel(image,cv2.CV_8U,1,0,ksize=3)
			if filterType == "6" :
				R, G, B = cv2.split(image)
				output1_R = cv2.equalizeHist(R)
				output1_G = cv2.equalizeHist(G)
				output1_B = cv2.equalizeHist(B)
				image = cv2.merge((output1_R, output1_G, output1_B))



		image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
		if train_test == str(FOLD):
			test_labels.append(label)
			test_data.append(image)
			# test_files.append(path_parts[-1])
		else:
			train_labels.append(label)
			train_data.append(image)

	# Prepare data for model
	print(
		f'\nNumber of training samples: {len(train_labels)} \n'
		f'Number of testing samples: {len(test_labels)}'
	)

	assert len(set(train_labels)) == len(set(test_labels)), (
		'Something went wrong. Some classes are only in train or test data.'
	)  # yapf: disable

	# convert the data and labels to NumPy arrays while scaling the pixel
	# intensities to the range [0, 255]
	train_data = np.array(train_data) / 255.0
	test_data = np.array(test_data) / 255.0
	train_labels_text = np.array(train_labels)
	test_labels_text = np.array(test_labels)

	num_classes = len(set(train_labels))

	# perform one-hot encoding on the labels
	lb = LabelBinarizer()
	lb.fit(train_labels_text)

	train_labels = lb.transform(train_labels_text)
	test_labels = lb.transform(test_labels_text)

	if num_classes == 2:
		train_labels = to_categorical(train_labels, num_classes=num_classes)
		test_labels = to_categorical(test_labels, num_classes=num_classes)

	trainX = train_data
	trainY = train_labels
	testX = test_data
	testY = test_labels
	print('Class mappings are:', lb.classes_)


	trainAug = ImageDataGenerator(
		rotation_range=10,
		fill_mode='nearest',
		horizontal_flip=True,
		vertical_flip=True,
		width_shift_range=0.1,
		height_shift_range=0.1
	
	)

	# Load the VGG16 network
	print('Compiling VGG16...')
	model = MODEL_FACTORY[MODEL_ID](
		input_size=(IMG_WIDTH, IMG_HEIGHT, 3),
		num_classes=num_classes,
		trainable_layers=TRAINABLE_BASE_LAYERS,
		log_softmax=LOG_SOFTMAX,
		hidden_size=HIDDEN_SIZE
	)

	metrics = Metrics((testX, testY), model)

	# compile model
	print('Compiling model...')
	opt = Adam(learning_rate=LR, decay=LR / EPOCHS)
	loss = (
		tf.keras.losses.CategoricalCrossentropy() if not LOG_SOFTMAX else (
			lambda labels, targets: tf.reduce_mean(
				tf.reduce_sum(
					-1 * tf.math.multiply(tf.cast(labels, tf.float32), targets),
					axis=1
				)
			)
		)
	)


	model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])

	print(f'Model has {model.count_params()} parameters')
	print(f'Model summary {model.summary()}')

	# train the head of the network
	print('Starting training model...')
	model.fit(trainAug.flow(x=trainX, y=trainY, batch_size=BATCH_SIZE),epochs=EPOCHS)
	print('Done')

	#model_ = Model(inputs=model.input, outputs=model.get_layer(index=-5).output)
#	print(model_.summary())
	csv_data = []
  
  # if filter_types_applied == "1" :
  #   print("blur filter aplicated")
  #   filter_types_applied_string += "filter2D"
	for filterType in filter_types_applied:
		if filterType == "0" :
			filter_types_applied_string += "noFilter"
		if filterType == "1" :
			print("2d filter aplicated")
			filter_types_applied_string += "filter2D"
		if filterType == "2" :
			print("blur filter aplicated")
			filter_types_applied_string += "blur"
		if filterType == "3" :
			print("medianBlur filter aplicated")
			filter_types_applied_string += "medianBlur"
		if filterType == "4" :
			print("Laplacian filter aplicated")
			filter_types_applied_string += "Laplacian"
		if filterType == "5" :
			print("Sobel filter aplicated")
			filter_types_applied_string += "Sobel"
		if filterType == "6" :
			print("equalizeHist filter aplicated")
			filter_types_applied_string += "equalizeHist"
   
		
	for imagePath in imagePaths:
		images=[]
		path_parts = imagePath.split(os.path.sep)
		train_test = path_parts[-3][-1]
		label = path_parts[-2]

		image = cv2.imread(imagePath)

		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    

		for filterType in filter_types_applied:
			if filterType == "0" :
				image = image
			if filterType == "1" :
				kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
				image = cv2.filter2D(image, -1, kernel)
			if filterType == "2" :
				ksize = (3, 3)
				image = cv2.blur(image, ksize)
			if filterType == "3" :
				image = cv2.medianBlur(image,3)
			if filterType == "4" :
				image = cv2.Laplacian(image,cv2.CV_8U)
			if filterType == "5" :
				image = cv2.Sobel(image,cv2.CV_8U,1,0,ksize=3)
			if filterType == "6" :
				R, G, B = cv2.split(image)
				output1_R = cv2.equalizeHist(R)
				output1_G = cv2.equalizeHist(G)
				output1_B = cv2.equalizeHist(B)
				image = cv2.merge((output1_R, output1_G, output1_B))

		image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
		images.append(image)

		X = np.array(images) / 255.0 #convierto de lista a numpy
		test_X = X.astype('float32')
		test_X = test_X / 255.

		#print(contador)
		#contador = contador + 1
		predicted_classes = model.predict(test_X)
		test_array = np.append(predicted_classes[0],imagePath)
		test_array = np.append(test_array,label)
		if train_test == str(FOLD):
			test_array = np.append(test_array,'test')
		else:
			test_array = np.append(test_array,'train')
		test_array = np.array(test_array);
		csv_data.append(test_array)


	df = pd.DataFrame(csv_data)
	print('/content/drive/My Drive/test_data/fold_'+str(FOLD)+'/'+'/result_model_'+filter_types_applied_string+'.csv')
	if not os.path.exists('/content/drive/My Drive/test_data/fold_'+str(FOLD)+'/'):
		os.makedirs('/content/drive/My Drive/test_data/fold_'+str(FOLD)+'/')
	#df.to_csv('test_data/fold_'+str(FOLD)+'/'+'/result_model_filter_'+filter_types_applied_string+'.csv')
	df.to_csv('/content/drive/My Drive/test_data/fold_'+str(FOLD)+'/'+'/result_model_'+filter_types_applied_string+'.csv')
	del model
	gc.collect()


In [ ]:
filter = ["1","2","3"]
filter2 = ["4", "5","6"]
for x in filter2:
    naive(x)

In [ ]:
naive("0")

Loading images...
selected fold: 4

Number of training samples: 1595 
Number of testing samples: 587
Class mappings are: ['covid' 'pneumonia' 'regular']
Compiling VGG16...
Compiling model...
Model has 14747971 parameters
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
____________________________________

In [ ]:
gc.collect()

88